In [2]:
# 1. IMPORT LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import sqlite3
import pandas as pd

# Conectar a la base de datos
conn = sqlite3.connect('/Users/dicordova/Proyecto-FDS/data/nueva_base.db')

# Leer las tablas desde la base de datos
df_vivienda = pd.read_sql("SELECT * FROM df_renamed_vivienda", conn)
df_hogar = pd.read_sql("SELECT * FROM df_renamed_hogar", conn)

# Cerrar la conexión (opcional aquí, pero buena práctica)
conn.close()

# Verificar contenido
print(df_vivienda.head())
print(df_hogar.head())


   provincia  canton_id  nro_vivienda  tipo_via tipo_vivienda  ocup_viv_part  \
0  Pichincha          1           1.0         1      Mediagua            1.0   
1  Pichincha          1           2.0         3    Casa/villa            1.0   
2  Pichincha          1           3.0         4    Casa/villa            1.0   
3  Pichincha          1           4.0         7    Casa/villa            1.0   
4  Pichincha          1           5.0         6    Casa/villa            1.0   

   ocup_viv_colect  mat_techo  estado_techo  mat_paredes  ...   id_vivienda  \
0              NaN        2.0           1.0          2.0  ...  5.010000e+09   
1              NaN        1.0           1.0          2.0  ...  5.010000e+09   
2              NaN        1.0           1.0          2.0  ...  5.010000e+09   
3              NaN        3.0           2.0          2.0  ...  5.010000e+09   
4              NaN        2.0           1.0          2.0  ...  5.010000e+09   

   total_fallecidos  total_emigrantes  total

In [4]:

# Merge the DataFrames on a common column
merged_df = pd.merge(df_vivienda, df_hogar, on=['nro_vivienda', 'canton_id'], how='inner')
print(merged_df.head())

  provincia_x  canton_id  nro_vivienda  tipo_via tipo_vivienda  ocup_viv_part  \
0   Pichincha          1           1.0         1      Mediagua            1.0   
1   Pichincha          1           1.0         1      Mediagua            1.0   
2   Pichincha          1           1.0         1      Mediagua            1.0   
3   Pichincha          1           1.0         1      Mediagua            1.0   
4   Pichincha          1           1.0         1      Mediagua            1.0   

   ocup_viv_colect  mat_techo  estado_techo  mat_paredes  ...  total_hombres  \
0              NaN        2.0           1.0          2.0  ...              2   
1              NaN        2.0           1.0          2.0  ...              0   
2              NaN        2.0           1.0          2.0  ...              1   
3              NaN        2.0           1.0          2.0  ...              2   
4              NaN        2.0           1.0          2.0  ...              1   

   total_mujeres  total_personas

Podemos ver que las variables son categóricas, con valores no necesariamente ordenados por importancia, es decir, debemos reemplazar estos valores para poder entender la relación entre categorías de mejor manera. Sin embargo, primero debemos investigar si vale la pena quedarse con todas las columnas y qué hacer con los valores nulos. 

Combinamos ambas tablas:

In [5]:
print("Columnas actuales:")
for col in merged_df.columns:
    print(col)

Columnas actuales:
provincia_x
canton_id
nro_vivienda
tipo_via
tipo_vivienda
ocup_viv_part
ocup_viv_colect
mat_techo
estado_techo
mat_paredes
estado_paredes
mat_piso
estado_piso
tipo_agua
fuente_agua
tipo_sanitario
electricidad_red
otra_fuente_elec
eliminacion_basura
nro_cuartos
gasto_compartido
nro_hogares
zona_x
canton_x
id_vivienda_x
total_fallecidos
total_emigrantes
total_personas_x
ocup_viv_part_r
nro_cuartos_r
deficit_habitacional
registro_imputado_vivienda
mat_techo_labels
mat_techo_desc
provincia_y
nro_hogar
nro_dormitorios
espacio_cocina
tiene_inodoro
tiene_ducha
combustible_cocina
fuente_agua_bebida
separa_basura_organica
separa_basura_animales
separa_basura_reciclaje
tiene_perros
nro_perros
tiene_gatos
nro_gatos
tenencia_vivienda
tiene_telefono_fijo
tiene_celular
tiene_tv_pagada
tiene_internet
tiene_computadora
tiene_refrigeradora
tiene_lavadora
tiene_secadora
tiene_microondas
tiene_extractora_olores
tiene_auto
tiene_moto
fallecidos_ultimos_3_anios
nro_fallecidos
emigrantes_

Sabemos por el dataset que las siguientes variables del dataset Hogar (Table 1, con terminación _x) son derivadas, que por recomendación del INEC deberían conservarse. 

- Área urbana o rural
- Cantón
- Identificador de la vivienda
- Total de fallecidos de la vivienda
- Total de emigrantes de la vivienda
- Total de personas de la vivienda

In [6]:
#We take a look at the missing values
missing_counts = merged_df.isnull().sum().sort_values(ascending=False)
print("\nMissing values by column:\n", missing_counts)


Missing values by column:
 ocup_viv_colect            7699027
nro_emigrantes             7583299
nro_fallecidos             7356313
nro_gatos                  5485020
nro_perros                 3423834
                            ...   
zona_x                           0
tipo_vivienda                    0
tipo_via                         0
nro_vivienda                     0
registro_imputado_hogar          0
Length: 76, dtype: int64


In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

# Total number of rows
total_rows = merged_df.shape[0]

# Calculate missing values in percentage
missing_percent = (merged_df.isnull().sum() / total_rows * 100).sort_values(ascending=False)

print("\nMissing values by column (percentage):\n", missing_percent)



Missing values by column (percentage):
 ocup_viv_colect                 99.859777
nro_emigrantes                  98.358733
nro_fallecidos                  95.414625
nro_gatos                       71.143129
nro_perros                      44.408637
deficit_habitacional            17.320429
tipo_sanitario                  17.320429
mat_techo_desc                  17.320429
mat_techo_labels                17.320429
nro_cuartos_r                   17.320429
gasto_compartido                17.320429
nro_cuartos                     17.320429
eliminacion_basura              17.320429
otra_fuente_elec                17.320429
electricidad_red                17.320429
nro_hogares                     17.320429
fuente_agua                     17.320429
tipo_agua                       17.320429
mat_techo                       17.320429
estado_techo                    17.320429
mat_paredes                     17.320429
mat_piso                        17.320429
estado_paredes                  17.

In [8]:
# We can see that the columns 'nro_hogar', 'nro_vivienda', and 'canton_id' have a lot of missing values.
# we can also see that some columns have the same information. We will drop the columns that are not derivated. 
merged_df.drop(columns=['total_fallecidos', 'total_emigrantes', 'total_personas_y', 'id_vivienda_y'], inplace=True)

In [9]:
# We know, from the data exploration, that the variables in the dataset HOGAR or Table 1 were not extracted from empty houses or houses in construction
# We can drop the rows where the variable ocup_viv_part is 3.0 or 4.0
filtered_df = merged_df[~merged_df['ocup_viv_part_r'].isin([3.0, 4.0])]


In [10]:
#We check null values again
# Total number of rows
total_rows = merged_df.shape[0]

# Calculate missing values in percentage
missing_percent = (filtered_df.isnull().sum() / total_rows * 100).sort_values(ascending=False)

print("\nMissing values by column (percentage):\n", missing_percent)


Missing values by column (percentage):
 ocup_viv_colect                 90.090778
nro_emigrantes                  88.747766
nro_fallecidos                  86.103716
nro_gatos                       64.270170
nro_perros                      40.101621
tipo_sanitario                   7.551430
mat_techo_desc                   7.551430
mat_techo_labels                 7.551430
deficit_habitacional             7.551430
nro_cuartos_r                    7.551430
nro_hogares                      7.551430
nro_cuartos                      7.551430
eliminacion_basura               7.551430
otra_fuente_elec                 7.551430
electricidad_red                 7.551430
gasto_compartido                 7.551430
fuente_agua                      7.551430
estado_paredes                   7.551430
mat_techo                        7.551430
tipo_agua                        7.551430
mat_paredes                      7.551430
estado_techo                     7.551430
mat_piso                         7.

Gracias a la exclusión de hogares desocupados o en construcción los porcentajes de valores nulos se redujo en más del 10% para muchas variables.

In [11]:
# Continuamos con la limpieza de datos. Vamos a analizar si hay columnas con valores duplicados
# Check for duplicate rows
duplicates = filtered_df[filtered_df.duplicated()]
print("\nDuplicate rows:\n", duplicates)


Duplicate rows:
 Empty DataFrame
Columns: [provincia_x, canton_id, nro_vivienda, tipo_via, tipo_vivienda, ocup_viv_part, ocup_viv_colect, mat_techo, estado_techo, mat_paredes, estado_paredes, mat_piso, estado_piso, tipo_agua, fuente_agua, tipo_sanitario, electricidad_red, otra_fuente_elec, eliminacion_basura, nro_cuartos, gasto_compartido, nro_hogares, zona_x, canton_x, id_vivienda_x, total_personas_x, ocup_viv_part_r, nro_cuartos_r, deficit_habitacional, registro_imputado_vivienda, mat_techo_labels, mat_techo_desc, provincia_y, nro_hogar, nro_dormitorios, espacio_cocina, tiene_inodoro, tiene_ducha, combustible_cocina, fuente_agua_bebida, separa_basura_organica, separa_basura_animales, separa_basura_reciclaje, tiene_perros, nro_perros, tiene_gatos, nro_gatos, tenencia_vivienda, tiene_telefono_fijo, tiene_celular, tiene_tv_pagada, tiene_internet, tiene_computadora, tiene_refrigeradora, tiene_lavadora, tiene_secadora, tiene_microondas, tiene_extractora_olores, tiene_auto, tiene_moto, fa

CONCLUSIONES

No tenemos valores duplicados. Sin embargo, hay columnas que tienen información similar y debemos elegir con cuál nos quedamos. Alternativamente, mantenemos todas las columnas y sólo elegimos algunas para hacer la modelación. 

Vamos a elegir qué tipo de método usamos para los datos faltantes. 

In [12]:
# We are going to drop columns whose importance is low according to our previous analysis
filtered_df_reduced = filtered_df.drop(columns=['nro_cuartos', 'nro_dormitorios', 'nro_perros', 'nro_gatos', 'separa_basura_organica', 'separa_basura_animales', 'persona_no_mencionada', 'nro_fallecidos'])
print("\nDataFrame after dropping columns:\n", filtered_df_reduced.head())


DataFrame after dropping columns:
   provincia_x  canton_id  nro_vivienda  tipo_via tipo_vivienda  ocup_viv_part  \
0   Pichincha          1           1.0         1      Mediagua            1.0   
1   Pichincha          1           1.0         1      Mediagua            1.0   
2   Pichincha          1           1.0         1      Mediagua            1.0   
3   Pichincha          1           1.0         1      Mediagua            1.0   
4   Pichincha          1           1.0         1      Mediagua            1.0   

   ocup_viv_colect  mat_techo  estado_techo  mat_paredes  estado_paredes  \
0              NaN        2.0           1.0          2.0             1.0   
1              NaN        2.0           1.0          2.0             1.0   
2              NaN        2.0           1.0          2.0             1.0   
3              NaN        2.0           1.0          2.0             1.0   
4              NaN        2.0           1.0          2.0             1.0   

   mat_piso  estado_

In [13]:
categorias_columnas = {
    'espacio_cocina': {1.0: "Sí", 2.0: "No"},
    'tiene_inodoro': {1.0: "De uso exclusivo del hogar", 2.0: "Compartido con varios hogares", 3.0: "No tiene"},
    'tiene_ducha': {1.0: "De uso exclusivo del hogar", 2.0: "Compartido con varios hogares", 3.0: "No tiene"},
    'combustible_cocina': {
        1.0: "Gas de tanque o cilindro", 2.0: "Gas centralizado (por tubería)", 3.0: "Electricidad", 4.0: "Leña o carbón",
        5.0: "Biogás (residuos vegetales y/o animales, etc.)", 6.0: "Otro (Ej: gasolina, kerex, diésel, etc.)", 7.0: "Ninguno (no cocina)"
    },
    'fuente_agua_bebida': {
        1.0: "La beben, tal como llega al hogar", 2.0: "La compran (agua envasada en bidón, botella o funda)",
        3.0: "La hierven", 4.0: "Le ponen cloro", 5.0: "La filtran (colocan filtros en el grifo o usan purificadores)",
        6.0: "Realizan otro tratamiento"
    },
    'separa_basura_organica': {1.0: "Sí", 2.0: "No"},
    'separa_basura_animales': {1.0: "Sí", 2.0: "No"},
    'separa_basura_reciclaje': {1.0: "Sí", 2.0: "No"},
    'tiene_perros': {1.0: "Sí", 2.0: "No"},
    'tiene_gatos': {1.0: "Sí", 2.0: "No"},
    'tenencia_vivienda': {
        1.0: "Propia y totalmente pagada", 2.0: "Propia y la está pagando", 3.0: "Propia (regalada, donada, heredada o por posesión)",
        4.0: "Arrendada/anticresis", 5.0: "Prestada o cedida (no paga)", 6.0: "Por servicios"
    },
    'tiene_telefono_fijo': {1.0: "Sí", 2.0: "No"},
    'tiene_celular': {1.0: "Sí", 2.0: "No"},
    'tiene_tv_pagada': {1.0: "Sí", 2.0: "No"},
    'tiene_internet': {1.0: "Sí", 2.0: "No"},
    'tiene_computadora': {1.0: "Sí", 2.0: "No"},
    'tiene_refrigeradora': {1.0: "Sí", 2.0: "No"},
    'tiene_lavadora': {1.0: "Sí", 2.0: "No"},
    'tiene_secadora': {1.0: "Sí", 2.0: "No"},
    'tiene_microondas': {1.0: "Sí", 2.0: "No"},
    'tiene_extractora_olores': {1.0: "Sí", 2.0: "No"},
    'tiene_auto': {1.0: "Sí", 2.0: "No"},
    'tiene_moto': {1.0: "Sí", 2.0: "No"},
    'fallecidos_ultimos_3_anios': {1.0: "Sí", 2.0: "No", 9.0: "Se ignora"},
    'emigrantes_desde_2010': {1.0: "Sí", 2.0: "No", 9.0: "Se ignora"},
    'persona_no_mencionada': {1.0: "Sí", 2.0: "No", 9.0: "Se ignora"},
    'zona': {1.0: "Área Urbana", 2.0: "Área Rural"},
    'nro_cuartos_con_sala_comedor': {
        0.0: "Ninguno", 1.0: "Un dormitorio", 2.0: "Dos dormitorios", 3.0: "Tres dormitorios",
        4.0: "Cuatro dormitorios", 5.0: "Cinco dormitorios", 6.0: "Seis o más dormitorios"
    },
    'registro_imputado_hogar': {1.0: "Sí", 2.0: "No"},
    'tipo_via': {
        1.0: "Calle", 2.0: "Avenida", 3.0: "Carretera", 4.0: "Pasaje",
        5.0: "Callejón", 6.0: "Sendero", 7.0: "Camino", 8.0: "Otro"
    },
    'tipo_vivienda': {
        1.0: "Casa/villa", 2.0: "Departamento en casa o edificio", 3.0: "Cuarto/s en casa de inquilinato",
        4.0: "Mediagua", 5.0: "Rancho", 6.0: "Covacha", 7.0: "Choza", 8.0: "Otra vivienda particular",
        9.0: "Hotel, pensión, residencial u hostal", 10.0: "Cuartel militar, policía o bomberos",
        11.0: "Centro de privación de libertad/cárcel", 12.0: "Hospital, clínica, etc.",
        13.0: "Convento o institución religiosa", 14.0: "Centro de acogida y protección para niñas/os y adolescentes",
        15.0: "Residencia de adultos mayores/Asilo de ancianos", 16.0: "Internado de estudiantes",
        17.0: "Campamento de trabajo", 18.0: "Otra vivienda colectiva", 19.0: "Sin vivienda"
    },
    'ocup_viv_part': {
        1.0: "Ocupada con personas presentes", 2.0: "Ocupada con personas ausentes", 3.0: "De temporada o vacacional",
        4.0: "Desocupada", 5.0: "En construcción"
    },
    'ocup_viv_colect': {1.0: "Con residentes habituales", 2.0: "Sin residentes habituales"},
    'mat_techo': {
        1.0: "Hormigón (losa, cemento)", 2.0: "Fibrocemento, asbesto (eternit, eurolit)",
        3.0: "Zinc, aluminio (lámina o plancha metálica)", 4.0: "Teja", 5.0: "Palma, paja u hoja", 6.0: "Otro material"
    },
    'estado_techo': {1.0: "Bueno", 2.0: "Regular", 3.0: "Malo"},
    'mat_paredes': {
        1.0: "Hormigón", 2.0: "Ladrillo o bloque", 3.0: "Panel prefabricado (yeso, fibrocemento, etc.)",
        4.0: "Adobe o tapia", 5.0: "Madera", 6.0: "Caña revestida o bahareque", 7.0: "Caña no revestida", 8.0: "Otro material"
    },
    'estado_paredes': {1.0: "Bueno", 2.0: "Regular", 3.0: "Malo"},
    'mat_piso': {
        1.0: "Duela, parquet, tablón o piso flotante", 2.0: "Cerámica, baldosa, vinil o porcelanato",
        3.0: "Mármol o marmetón", 4.0: "Ladrillo o cemento", 5.0: "Tabla sin tratar", 6.0: "Caña sin tratar",
        7.0: "Tierra", 8.0: "Otro material"
    },
    'estado_piso': {1.0: "Bueno", 2.0: "Regular", 3.0: "Malo"},
    'tipo_agua': {
        1.0: "Por tubería, dentro de la vivienda",
        2.0: "Por tubería, fuera de la vivienda pero dentro del edificio, lote o terreno",
        3.0: "Por tubería, fuera del edificio, lote o terreno",
        4.0: "No recibe agua por tubería, sino por otros medios"
    },
    'fuente_agua': {
        1.0: "Empresa Pública/Municipio", 2.0: "Juntas de Agua/Organizaciones comunitarias/GAD parroquial",
        3.0: "Pozo", 4.0: "Carro o tanquero repartidor",
        5.0: "Otras fuentes (río, vertiente, acequia, canal, grieta o agua lluvia)"
    },
    'tipo_sanitario': {
        1.0: "Inodoro o escusado, conectado a red pública de alcantarillado",
        2.0: "Inodoro o escusado, conectado a pozo séptico",
        3.0: "Inodoro o escusado, conectado a biodigestor",
        4.0: "Inodoro o escusado, conectado a pozo ciego",
        5.0: "Inodoro o escusado, con descarga directa al mar, río, lago o quebrada",
        6.0: "Letrina", 7.0: "No tiene"
    },
    'electricidad_red': {1.0: "Sí", 2.0: "No"},
    'otra_fuente_elec': {
        1.0: "Planta eléctrica (generador de luz)", 2.0: "Energía solar (panel fotovoltaico)",
        3.0: "Energía eólica (a partir del viento)", 4.0: "Otra fuente (desechos vegetales y animales)",
        5.0: "No dispone"
    },
    'eliminacion_basura': {
        1.0: "Por carro recolector", 2.0: "Por contenedor municipal", 3.0: "La arroja en terreno baldío",
        4.0: "La quema", 5.0: "La entierra", 6.0: "La arroja al río, acequia, canal o quebrada", 7.0: "De otra forma"
    },
    'gasto_compartido': {1.0: "Sí", 2.0: "No"},
    'ocup_viv_part_r': {
        1.0: "Ocupada", 2.0: "De temporada o vacacional", 3.0: "Desocupada", 4.0: "En construcción"
    },
    'nro_cuartos_r': {
        1.0: "Un cuarto", 2.0: "Dos cuartos", 3.0: "Tres cuartos", 4.0: "Cuatro cuartos",
        5.0: "Cinco cuartos", 6.0: "Seis o más cuartos"
    },
    'deficit_habitacional': {
        1.0: "Dignas o aceptables", 2.0: "Déficit cualitativo (Recuperables)", 3.0: "Déficit cuantitativo (Irrecuperables)"
    },
    'registro_imputado_vivienda': {1.0: "Sí", 2.0: "No"}
}


In [14]:
# 2. Apply mappings to all specified columns
for col in categorias_columnas.keys():
    if col in filtered_df_reduced.columns:
        # Create new column with '_label' suffix to keep both numeric and text versions
        filtered_df_reduced[f'{col}_label'] = filtered_df_reduced[col].map(categorias_columnas[col])
        
        # Alternatively, to replace the numeric values directly:
        # df[col] = df[col].map(categorias[col])

# 3. Verify the changes
# Check one example column
print("\nOriginal numeric values:")
print(filtered_df_reduced['tipo_vivienda'].value_counts(dropna=False))

print("\nMapped labeled values:")
print(filtered_df_reduced['tipo_vivienda_label'].value_counts(dropna=False))

# Check for missing values after mapping
print("\nMissing values after mapping:")
print(filtered_df_reduced[[f'{col}_label' for col in categorias_columnas.keys() if f'{col}_label' in filtered_df_reduced.columns]].isnull().sum())

# # 4. Optional: Quick visualization of one categorical variable
# import matplotlib.pyplot as plt

# if 'tipo_vivienda_label' in filtered_df_reduced.columns:
#     filtered_df_reduced['tipo_vivienda_label'].value_counts().plot(kind='bar', figsize=(12,6))
#     plt.title("Distribution of Housing Types")
#     plt.xticks(rotation=45, ha='right')
#     plt.tight_layout()
#     plt.show()


Original numeric values:
tipo_vivienda
Casa/villa                      4359875
Departamento                    1457640
Mediagua                         468464
Rancho                           379053
Cuarto en inquilinato            213779
Covacha                           48995
Choza                             10212
Otra vivienda particular           7101
Hotel/pensión/hostal               4509
Otra vivienda colectiva            2482
Institución religiosa              1629
Campamento de trabajo               933
Sin vivienda                        734
Hospital/clínica                    327
Asilo de ancianos                   324
Cuartel                             262
Centro de acogida (niños/as)        194
Internado                            94
Cárcel                               57
Name: count, dtype: int64

Mapped labeled values:
tipo_vivienda_label
NaN    6956664
Name: count, dtype: int64

Missing values after mapping:
espacio_cocina_label                     4693
tiene_inodor

Con este paso anterior, se crean columnas nuevas con el formato _label sin modificar los valores numericos de las columnas. Esto permite que no se cambie mi df original. 


In [20]:
print("Columnas actuales:")
print(filtered_df_reduced.columns.tolist())
print(filtered_df_reduced.dtypes)
print(filtered_df_reduced.head())

Columnas actuales:
['provincia_x', 'canton_id', 'nro_vivienda', 'tipo_via', 'tipo_vivienda', 'ocup_viv_part', 'ocup_viv_colect', 'mat_techo', 'estado_techo', 'mat_paredes', 'estado_paredes', 'mat_piso', 'estado_piso', 'tipo_agua', 'fuente_agua', 'tipo_sanitario', 'electricidad_red', 'otra_fuente_elec', 'eliminacion_basura', 'gasto_compartido', 'nro_hogares', 'zona_x', 'canton_x', 'id_vivienda_x', 'total_personas_x', 'ocup_viv_part_r', 'nro_cuartos_r', 'deficit_habitacional', 'registro_imputado_vivienda', 'mat_techo_labels', 'mat_techo_desc', 'provincia_y', 'nro_hogar', 'espacio_cocina', 'tiene_inodoro', 'tiene_ducha', 'combustible_cocina', 'fuente_agua_bebida', 'separa_basura_reciclaje', 'tiene_perros', 'tiene_gatos', 'tenencia_vivienda', 'tiene_telefono_fijo', 'tiene_celular', 'tiene_tv_pagada', 'tiene_internet', 'tiene_computadora', 'tiene_refrigeradora', 'tiene_lavadora', 'tiene_secadora', 'tiene_microondas', 'tiene_extractora_olores', 'tiene_auto', 'tiene_moto', 'fallecidos_ultimos

FEATURE ENGINEERING:

Las variables categoricas con valores sin un orden especifico se pueden modificar a traves de one-hot encoding, que es lo que haremos a continuacion

In [21]:
# Select the '_label' columns
label_cols = [f'{col}_label' for col in categorias_columnas.keys() if f'{col}_label' in filtered_df_reduced.columns]

# One-hot encode (text labels)
df_encoded = pd.get_dummies(filtered_df_reduced, columns=label_cols, prefix=label_cols)

print(df_encoded.head())

: 

In [16]:

# df_encoded.to_csv('/Users/dicordova/Proyecto-FDS/data/cotopaxi_df_processed.csv', index=False)

: 

Este es solo un ejemplo del trabajo de limpieza de datos que requerira este proyecto de ciencia de datos. 
En este notebook se inició con acciones de ingeniería de datos para una mejor comprensión del tipo de información que contiene. 

In [20]:
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect('/Users/dicordova/Proyecto-FDS/data/nueva_base.db')

# Save the DataFrame to the database
df_encoded.to_sql('df_encoded', conn, if_exists='replace', index=False)


# Close connection
conn.close()


In [ ]:
# import sqlite3
# import pandas as pd

# # Connect to the database
# conn = sqlite3.connect('/Users/dicordova/Proyecto-FDS/data/nueva_base.db')

# # Load the table
# df_encoded = pd.read_sql("SELECT * FROM df_encoded", conn)

# # # Save it to a CSV file in the data folder
# # df_encoded.to_csv('/Users/dicordova/Proyecto-FDS/data/data_processed.csv', index=False)

# # Close the connection
# conn.close()

# print("✅ df_encoded exported as data_processed.csv")


: 